# ETL Project Technical Report

## Team 5: Emerson Williams-Molett, Kerry Harp, Yonis Hassan

_______________________________________________________________________________

<img src = "images/annual-working-hours-per-worker.png" width=500 align="left"> 

_______________________________________________________________________________

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [9]:
import warnings
warnings.filterwarnings('ignore')

### Data Extraction

#### Sources:

Data Set 1:[World Happiness Report](https://worldhappiness.report/ed/2021/#appendices-and-data) (CSV file)

Data Set 2: Our World in Data [Are we working more than ever](https://ourworldindata.org/working-more-than-ever?) (CSV file)



#### Process:

Extract CSVs into DataFrames
    <ul>
    <li>World Happiness CSV has 11 items in the header (Country Name, Year, Life Ladder, Log GDP per capita, Social support, Healthy life expectancy at birth, Freedom to make life choices, generosity, Perceptions of corruption, Positive affect, Negative affect)</li>
   <li>Our World CSV has 4 items in the header (Entity, Code, Year, Average annual working hours per worker)</li>
    </ul>

In [2]:
#import the world_happiness csv
world_happiness = "data/world_happiness.csv"
world_happiness_df = pd.read_csv(world_happiness)
world_happiness_df.head()

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
0,Afghanistan,2008,3.724,7.370,0.451,50.80,0.718,0.168,0.882,0.518,0.258
1,Afghanistan,2009,4.402,7.540,0.552,51.20,0.679,0.190,0.850,0.584,0.237
2,Afghanistan,2010,4.758,7.647,0.539,51.60,0.600,0.121,0.707,0.618,0.275
3,Afghanistan,2011,3.832,7.620,0.521,51.92,0.496,0.162,0.731,0.611,0.267
4,Afghanistan,2012,3.783,7.705,0.521,52.24,0.531,0.236,0.776,0.710,0.268


In [3]:
#import the annual_working_hours_per_worker
annual_working_hours = "data/annual_working_hours_per_worker.csv"
annual_work_df = pd.read_csv(annual_working_hours)
annual_work_df.head()

,Entity,Code,Year,Average annual working hours per worker
0,Argentina,ARG,1950,2034.0000
1,Argentina,ARG,1951,2037.8667
2,Argentina,ARG,1952,2041.7408
3,Argentina,ARG,1953,2045.6223
4,Argentina,ARG,1954,2049.5112


_______________________________________________________________________________

### Data Cleanup

Happiness DataFrame
    <ul>
    <li>Kept the wanted columns (Country Name, Year, Life Ladder)</li>
   <li>Our World CSV has 4 items in the header (Entity, Code, Year, Average annual working hours per worker)</li>
    <li>Renamed column names.</li>
    </ul>

Work Hours DataFrame
    <ul>
    <li>Kept the wanted columns (Entity, Year, Average annual working hours per worker)</li>
   <li>Renamed column names</li>
    <li>Deleted all rows with the year < 2005 </li>
    <li>Remove decimal places from the work hours.</li>
    </ul>

In [4]:
world_happiness_df.head()

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
0,Afghanistan,2008,3.724,7.370,0.451,50.80,0.718,0.168,0.882,0.518,0.258
1,Afghanistan,2009,4.402,7.540,0.552,51.20,0.679,0.190,0.850,0.584,0.237
2,Afghanistan,2010,4.758,7.647,0.539,51.60,0.600,0.121,0.707,0.618,0.275
3,Afghanistan,2011,3.832,7.620,0.521,51.92,0.496,0.162,0.731,0.611,0.267
4,Afghanistan,2012,3.783,7.705,0.521,52.24,0.531,0.236,0.776,0.710,0.268


In [5]:
world_happiness_cols = ["Country name", "year", "Life Ladder"]
world_trans = world_happiness_df[world_happiness_cols].copy()
#world_trans.head()

# Rename the column headers
world_df = world_trans.rename(columns={'Country name': 'country', 'Life Ladder':'happiness_rank'})
world_df.head()


,country,year,happiness_rank
0,Afghanistan,2008,3.724
1,Afghanistan,2009,4.402
2,Afghanistan,2010,4.758
3,Afghanistan,2011,3.832
4,Afghanistan,2012,3.783


#### Transform annual_work DataFrame

In [6]:
# Create a filtered dataframe from specific columns
annual_cols = ["Entity", "Year", "Average annual working hours per worker"]
annual_transformed = annual_work_df[annual_cols].copy()

#rename the column headers
annual_transformed = annual_transformed.rename(columns={'Entity': 'country', 'Year': 'year', 
                                                        'Average annual working hours per worker': 'avg_work_hours'})


len(annual_transformed)
#keep the years 2005 and above 
annual_work_df = annual_transformed[annual_transformed['year'] >= 2005]
len(annual_work_df)


854

In [10]:
annual_work_df["avg_work_hours"] = annual_work_df["avg_work_hours"].astype(int)

In [8]:
annual_work_df.head()

,country,year,avg_work_hours
55,Argentina,2005,1761
56,Argentina,2006,1765
57,Argentina,2007,1780
58,Argentina,2008,1781
59,Argentina,2009,1742


_______________________________________________________________________________

### Connect and load database

Used Pandas to load the two dataframes into SQL database named ETL_5 (tables: world_happiness and annual_work_hours).

Confirm data has been added

Query the database

#### Create database connection

In [ ]:
#fill in the place holder with your username and password for postgres
conn = "<insert user name>:<insert password>@localhost:5432/etl_5"
engine = create_engine(f'postgresql://{conn}')

In [ ]:
#confirm tables
engine.table_names()

#### Load DataFrames into database

In [ ]:
world_df.to_sql(name='world_happiness', con=engine, if_exists='append', index=False)

In [ ]:
annual_work_df.to_sql(name='annual_work_hours', con=engine, if_exists='append', index=False)